<a href="https://colab.research.google.com/github/Elena-Gebel/Mathematical-methods-of-machine-learning/blob/main/Copy_of_matrix_decomp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy
import pandas as pd

from typing import Callable

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Напоминание*:
* Собственные числа и собственные векторы матрицы $\mathbf{A}$: пусть $E$ - векторное пространство над $K$, $A: E \longrightarrow E$, $\lambda \in K$ - собственное число $\mathbf{A}$, если $\exists \mathbf{u} \in E$, такое что $\mathbf{A} \mathbf{u} = \lambda \mathbf{u}$. В таком случае $\mathbf{u}$ - собственный вектор.
* Спектр и спектральный радиус $\mathbf{A}$: спектр $\Lambda(\mathbf{A})$- множество всех $\lambda$ - собственных чисел матрицы $\mathbf{A}$, спектральный радиус - $\rho(\mathbf{A}) = \max_{\lambda \in \Lambda(\mathbf{A})} |\lambda |$.
* Сопряжённая матрица: матрица $\mathbf{A}^{*}$, полученная из $\mathbf{A}$ заменой каждого элемента на его комплексно-сопряжённый и транспонированием: $\mathbf{A}^{*} = \mathbf{\overline{A}}^{T}$.


**Норма матрицы и число обусловленности матрицы**

Пусть $\mathbf{x} = (x_1, x_2, \; ... \; x_n) \in E$, и на $E$ введена норма $∥·∥_{E}$. Тогда согласованной с ней нормой матрицы $A$ - $∥·∥_{\alpha}$ называют норму $∥A∥_{\alpha}$:

\begin{equation}
∥ A ∥_{\alpha} = \sup_{\mathbf{x} \neq 0} \frac{∥ A \mathbf{x} ∥_{E}}{∥\mathbf{x}∥_{E}} \Longrightarrow ∥A\mathbf{x}∥_{E} \; \leq \; ∥A∥ ∥\mathbf{x}∥ \forall \mathbf{x}
\end{equation}


Классические примеры согласованных норм:

* $∥\mathbf{x}∥_{\infty} \equiv \max_{1 \leq i \leq n}|x_{i}| \; : \; ∥\mathbf{A}∥_{\infty} \equiv \max_{1 \leq i \leq n} (\sum_{j=1}^{n}|a_{ij}|)$

* $∥\mathbf{x}∥_{1} \equiv \sum_{i=1}^{n}|x_{i}| \; : \; ∥\mathbf{A}∥_{1} \equiv \max_{1 \leq j \leq n} (\sum_{i=1}^{n}|a_{ij}|)$

* спектральная норма: $∥\mathbf{x}∥_{2} \equiv \sqrt{\sum_{i=1}|x_{i}|^2} \; : \; ∥\mathbf{A}∥_{2} \equiv \sup_{\mathbf{x} \neq 0} \frac{∥\mathbf{A}\mathbf{x}∥_{2}}{∥\mathbf{x}∥_{2}} = \sqrt{\lambda_{max}(A^{*} A)} = \sigma_{max}(A)$

Интерпретация спектральной нормы: максимальная величина, на которую $\mathbf{A}$ масштабирует единичную сферу в $E$.

Где могут применяться матричные нормы в машинном обучении? Контроль "худших сценариев" при линейных отображениях в машинном обучении:

(Рассмотрим интересный случай спектральной нормы)
* Регуляризация и предотвращение переобучения: $\mathit{L}_{reg} = \mathit{L}_{base} + \lambda \sum_{l = 1}^{\text{L}} ∥ W^{l} ∥_{2}^2 $ - помогают понизить чувствительность к возмущениям в обучающей выборке;

* При объяснении эффекта затухающих градиентов в обучении рекуррентных нейронных сетей: многократное применение матрицы весов к входам приводит к тому, что если $∥ W ∥_{2} << 1$  градиент затухает, а если $∥ W ∥_{2} >> 1$ - взрывается;

* Низкоранговые матричные разложения в контексте рекомендательных систем: при разложении (сверх-)большой матрицы $R \approx U V^{T}$. Ранг матрицы связан с числом ненулевых сингулярных значений. При обучении моделей часто регуляризуют ядерную норму (nuclear norm) $∥R∥_{*} = \text{trace}(\sqrt{A^{*} A}) = \sum_{i = 1}^{\min{(m, n)}} \sigma_{i}(R)$ - спектральная норма является двойственной нормой к ядерной и часто применяется в анализе алгоритмов.

In [ ]:
def norm_inf(A):
    return np.max(np.sum(np.abs(A), axis = 1))

def  norm_1(A):
    return np.max(np.sum(np.abs(A), axis = 0))

def norm_2(A):
    eigval, _ = np.linalg.eig(np.dot(A.conjugate().T, A))
    return np.sqrt(np.max(eigval))

#### Свойства операторных норм:

* $∥\mathbf{A} \mathbf{B} ∥ \leq ∥\mathbf{A}∥ \cdot ∥\mathbf{B} ∥$

    Для $\mathbf{A}$ - матрицы n на n:

* $n^{-1/2} ∥\mathbf{A}∥_{2} \leq ∥\mathbf{A}∥_{1} \leq n^{1/2} ∥\mathbf{A}∥_{2}$

* $n^{-1/2} ∥\mathbf{A}∥_{2} \leq ∥\mathbf{A}∥_{\inf} \leq n^{1/2} ∥\mathbf{A}∥_{2}$

* $n^{-1} ∥\mathbf{A}∥_{\inf} \leq ∥\mathbf{A}∥_{1} \leq n^{1/2} ∥\mathbf{A}∥_{\inf}$


In [ ]:
B = np.array([[1, 2, -1, 5],
              [2, 4, 3, 2],
              [-1, -7, 3, 22],
              [2, 1, 1, 10]])

C = np.random.randint(low = -5, high = 5, size = B.shape)

print('inf norm: custom ', norm_inf(B), ' numpy:', np.linalg.norm(B, ord = np.inf))
print('1-st norm: custom ', norm_1(B), ' numpy:', np.linalg.norm(B, ord = 1))
print('2-nd norm: custom ', norm_2(B), ' numpy:', np.linalg.norm(B, ord = 2))

# Для pytorch: аналоги норм - torch.linalg.matrix_norm(B, ord = np.inf), или 'fro', 1, 2, ...

# Выборочная проверка условий:

print(f'Norm of product vs. product of norms for inf-norm: {norm_inf(np.dot(B, C))} <= {norm_inf(B) * norm_inf(C)}')
print(f'Norm of product vs. product of norms for 1-norm: {norm_1(np.dot(B, C))} <= {norm_1(B) * norm_1(C)}')
print(f'Norm of product vs. product of norms for 2-norm: {norm_2(np.dot(B, C))} <= {norm_2(B) * norm_2(C)}')

inf norm: custom  33  numpy: 33.0
1-st norm: custom  39  numpy: 39.0
2-nd norm: custom  25.501404264314182  numpy: 25.501404264314182
Norm of product vs. product of norms for inf-norm: 287 <= 462
Norm of product vs. product of norms for 1-norm: 209 <= 507
Norm of product vs. product of norms for 2-norm: 187.00311723530803 <= 225.5191261368259


Число обусловленности матрицы: связано c решением СЛАУ $\mathbf{A} \mathbf{x} = \mathbf{b}$ и показывает, как решение уравнения будет вести себя при неточностях в правой части $\mathbf{\hat{b}} = \mathbf{b} + \Delta \mathbf{b}$ и система принимает вид: $\mathbf{A} \mathbf{\hat{x}} = \mathbf{\hat{b}}$, то есть:

\begin{equation}
\mathbf{A} \mathbf{x} + \mathbf{A} \Delta \mathbf{x} = \mathbf{b} + \Delta \mathbf{b}   
\end{equation}

Наша задача - оценка относительной ошибки решения уравнения $(∥\Delta \mathbf{x}∥ \div ∥\mathbf{x}∥)$ к ошибке его правой части $(∥\Delta \mathbf{b}∥ \div ∥\mathbf{b}∥)$

\begin{aligned}
\frac{∥\Delta \mathbf{x}∥ \ ∥\mathbf{x}∥}{∥\Delta \mathbf{b}∥ \ ∥\mathbf{b}∥} &= \frac{∥\Delta \mathbf{x}∥ ∥\mathbf{b}∥}{∥\Delta \mathbf{b}∥ \ ∥\mathbf{x}∥} = \textit{    }\\[10pt]
&= \frac{∥\mathbf{A}^{-1} \Delta \mathbf{b}∥ ∥\mathbf{A} \mathbf{x}∥}{∥\Delta \mathbf{b}∥ \ ∥\mathbf{x}∥} \leq \textit{    }\\[10pt]
&\leq \frac{∥\mathbf{A}^{-1}∥ ∥\Delta \mathbf{b}∥ ∥\mathbf{A}∥ ∥\mathbf{x}∥}{∥\Delta \mathbf{b}∥ \ ∥\mathbf{x}∥} = ∥\mathbf{A}^{-1}∥ ∥\mathbf{A}∥ = cond(A)
\end{aligned}

Таким образом, число обусловленности позволяют получить верхнюю оценку вычисленного решения: для плохо обусловленных матриц малые возмущения правых частей СЛАУ приводят к значительным погрешностям решения. На практике обычно используется норма $∥A∥_{2}$.

Вектор невязок: $\mathbf{r} = \mathbf{b} - \mathbf{A}\mathbf{\hat{x}} = -\Delta \mathbf{b}$. Связан с методом (например, численным) решения СЛАУ.

\begin{equation}
\frac{∥ \Delta \mathbf{x} ∥}{∥\mathbf{x}∥} \leq cond(\mathbf{A}) \frac{∥ \mathbf{r} ∥}{∥\mathbf{b}∥}
\end{equation}

Отсюда следует, что небольшая ошибка решения уравнения соответствует небольшому вектору невязки только если $\mathbf{A}$ - хорошо обусловлена.

In [ ]:
def cond(A, norm: Callable):
    '''
    Для работы с прямоугольными матрицами: вместо обратной матрицы из np.linalg.inv берём псевдо-обратную np.linalg.pinv
    '''
    return norm(np.linalg.pinv(A)) * norm(A)

Рассмотрим пример простой СЛАУ с двумя переменными:

In [ ]:
A = np.array([[4.1, 2.8],
              [9.7, 6.6]])
b = np.array([4.1, 9.7])
x = np.linalg.solve(A, b)
print(x)

[1. 0.]


Введём небольшое возмущение в правую часть уравнений:

In [ ]:
b_hat = b + np.array([0.01, 0])
x_pert = np.linalg.solve(A, b_hat)
print(x_pert)

[0.34 0.97]


In [ ]:
print(cond(A, norm_2))

1622.9993838565108


In [ ]:
residual = cond(A, norm_2) * np.linalg.norm(b - b_hat) / np.linalg.norm(b)
sol_error = np.linalg.norm(x - x_pert) / np.linalg.norm(x_pert)
print(f'solution error {sol_error} must be lower than residual {residual}')

solution error 1.1414407083492109 must be lower than residual 1.5411772226901603


Приложение числа обусловленности в задаче линейной регрессии: высокое число обусловленности означает, что среди признаков есть переменные с высокой взаимной корреляцией.

In [ ]:
df_salary = pd.read_csv("/content/drive/My Drive/MOMO_mk1/Salary_Data.csv")
df_salary.head()

,YearsExperience,Age,Salary
0,1.1,21.0,39343
1,1.3,21.5,46205
2,1.5,21.7,37731
3,2.0,22.0,43525
4,2.2,22.2,39891


In [ ]:
arr_features = df_salary.to_numpy() #[['YearsExperience', 'Age', 'Salary']]
arr_features_randomized = np.random.uniform(size=arr_features.shape)

In [ ]:
print(f'Condition number for the matrix is {cond(arr_features, norm_2)}')
print(f'Condition number for the random matrix is {cond(arr_features_randomized, norm_2)}')

Condition number for the matrix is (112480.70719841136+0j)
Condition number for the random matrix is (3.7036084052171843+0j)


Рассмотрим нормальные матрицы: $A^{*}A = A A^{*}$, что для вещественных матриц выражается как $A^{T}A = A A^{T}$. Интересны тем, что диагоализуются (их спектральной теоремы): $A = UDU^{*}$, где $U$ - унитарная матрица ($U^{*} U = U U^{*} = I$).

\begin{equation}
cond(A) = \frac{\sigma_{max}(A)}{\sigma_{min}(A)} = \frac{|\lambda_{max}(A)|}{|\lambda_{min}(A)|}
\end{equation}

In [ ]:
A = np.array([[1,  0, 7],
              [0, -1, 0],
              [7,  0, 2]])

def normal_matrix_cond(inp: np.ndarray) -> float:
    assert np.all(np.dot(inp.T, inp) == np.dot(inp, inp.T)), 'Input matrix is not normal'
    eigval, _ = np.linalg.eig(inp)
    return np.max(np.abs(eigval)) / np.min(np.abs(eigval))

print(cond(A, norm_2), normal_matrix_cond(A))


8.5178344238091 8.5178344238091


**LU-разложение**

В случае несингулярной $n \times n$ матрицы $\mathbf{A}$ (где $det(\mathbf{A}) \neq 0$) LU-разложение (с матрицей перестановки/поворота $\mathbf{P}$) имеет форму: $\mathbf{A} = \mathbf{P}\mathbf{L}\mathbf{U}$. В разложении $\mathbf{L}$ - нижняя треугольная матрица с единицами на главной диагонали, $\mathbf{U}$ - верхняя треугольная (невырожденная) матрица.  

\begin{equation}
A = P'LU \Longleftrightarrow P A = LU
\end{equation}


Матрица перестановки $\mathbf{P}'$ имеет обратную матрицу $\mathbf{P}$, также являющуюся матрицей перестановки. В каждом столбце и строке только одна единица, остальные элементы - нули.

Перестановка строк: левое умножение матрицы $\mathbf{A}$, получаем $\mathbf{P}\mathbf{A}$. Для i-ой строки: единица соответствует номеру строки исходной матрицы, которая перемещается на её i-ую строку.

\begin{equation}
\mathbf{A} = \begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
\end{bmatrix}, \; \text{и} \; \mathbf{P} =
\begin{bmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
1 & 0 & 0 \\
\end{bmatrix}.
\end{equation}

При перестановке строк получаем:
\begin{equation}
\mathbf{P}\mathbf{A} =
\begin{bmatrix}
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
a_{11} & a_{12} & a_{13} \\
\end{bmatrix}.
\end{equation}

Аналогично вводится перестановка столбцов $\mathbf{A}\mathbf{P}$.

Имея возможность разложить $\mathbf{A}$ как $\mathbf{P}\mathbf{L}\mathbf{U}$ мы построим рекурсивный алгоритм следующим образом:

1. Выбор $i$ - индекс строки в $\mathbf{U}$ с наибольшим по модулю значением; пусть $\mathbf{P}_{1}$ - матрица перестановки, меняющая $i$-ую строку на первую:

\begin{equation}
\mathbf{P}_{1} = \begin{bmatrix}
0_{1(i-1)} & 1 & 0_{1(n-i)} \\
I_{(i-1)(i-1)} & 0 & 0_{(i-1)(n-i)} \\
I_{(n-i)(i-1)} & 0 & I_{(n-i)(n-i)} \\
\end{bmatrix} =
\begin{bmatrix}
0 & ... & 0 & 1 & 0 & ... & 0 \\
1 & ... & 0 & 0 & 0 & ... & 0 \\
\vdots & \ddots & \vdots & \vdots & \vdots & ... & \vdots \\
0 & ... & 1 & 0 & 0 & ... & 0 \\
0 & ... & 0 & 0 & 1 & ... & 0 \\
\vdots & \ddots & \vdots & \vdots & \vdots & ... & \vdots \\
0 & ... & 0 & 0 & 0 & ... & 1 \\
\end{bmatrix}.
\end{equation}

2. Обозначим $\mathbf{\hat{A}}$ повёрнутую матрицу $\mathbf{A}$: $\mathbf{\hat{A}} = \mathbf{P}\mathbf{A}$ и введём матрицу поворота $\mathbf{P}_2$, оставляющую 1-ую строку неизменной, но переставляющую остальные ряды:

\begin{equation}
\mathbf{P}_{1} = \begin{bmatrix}
1 & \mathbf{0} \\
\mathbf{0} & \mathbf{P}_{22} \\
\end{bmatrix}
\end{equation}

Рассмотрим полную матрицу поворота $\mathbf{P} = \mathbf{P_2}\mathbf{P_1}$ - произведение отдельных матриц поворота. $\mathbf{P}\mathbf{A} = \mathbf{P_2}\mathbf{P_1}\mathbf{A} = \mathbf{P_2}\mathbf{\hat{A}} $

3. Используя разложение на множители итоговую PLU - факторизацию можно рассмотреть в блочной форме:


\begin{aligned}
\mathbf{P}\mathbf{A} &= \mathbf{L}\mathbf{U} \\
\mathbf{P}_2\mathbf{\hat{A}} &= \mathbf{L}\mathbf{U} \\
\begin{bmatrix}
1 & \mathbf{0} \\
\mathbf{0} & \mathbf{P}_{22} \\
\end{bmatrix}
\begin{bmatrix}
\overline{a}_{11} & \mathbf{\overline{a}}_{11} \\
\mathbf{\overline{a}}_{21} & \mathbf{\overline{A}}_{22} \\
\end{bmatrix} &=
\begin{bmatrix}
1 & \mathbf{0} \\
\mathbf{\mathit{l}}_{21} & \mathbf{L}_{22} \\
\end{bmatrix}
\begin{bmatrix}
u_{11} & \mathbf{u}_{11} \\
\mathbf{0} & \mathbf{U}_{22} \\
\end{bmatrix}
 \\
\begin{bmatrix}
\overline{a}_{11} & \mathbf{\overline{a}_{11}} \\
\mathbf{P}_{22} \mathbf{\overline{a}}_{21} & \mathbf{P}_{22} \mathbf{\overline{A}}_{22} \\
\end{bmatrix} &=
\begin{bmatrix}
u_{11} & \mathbf{u_{11}} \\
u_{11}\mathbf{\mathit{l}}_{21} & (\mathbf{\mathit{l}}_{21}\mathbf{u_{11}} + \mathbf{L}_{22}\mathbf{U}_{22}) \\
\end{bmatrix}\\
\end{aligned}

4. Приравниваем элементы матриц и получаем уравнения:

\begin{aligned}
\overline{a}_{11} &= u_{11} \\
\overline{\mathbf{a}}_{12} &= \mathbf{u}_{12} \\
P_{22} \overline{\mathbf{a}}_{21} &= u_{11} \mathbf{\mathit{l}}_{21} \\
P_{22} \overline{\mathbf{A}}_{22} &= \mathbf{u}_{12} \mathbf{\mathit{l}}_{21} + \mathbf{L}_{22}\mathbf{U}_{22}\\
\end{aligned}

5. Переписываем последнее уравнение с учётом первых трёх и используем дополнение Шура $\mathbf{S}_{22} = (\overline{\mathbf{A}}_{22} - \overline{\mathbf{a}}_{21}(\overline{a}_{11})^{-1}\overline{\mathbf{a}}_{12} )$:
\begin{equation}
\mathbf{P}_{22} (\overline{\mathbf{A}}_{22} - \overline{\mathbf{a}}_{21}(\overline{a}_{11})^{-1}\overline{\mathbf{a}}_{12} ) = \mathbf{P}_{22} \mathbf{S}_{22} =  \mathbf{L}_{22}\mathbf{U}_{22}
\end{equation}

6. Далее, по рекурсии раскладываем $\mathbf{S}_22$ на $\mathbf{L}_22$, $\mathbf{U}_22$ и $\mathbf{P}_22$.

7. Решаем для первых строчек и столбцов $\mathbf{L}$ и $\mathbf{U}$:

\begin{aligned}
\overline{a}_{11} &= u_{11} \\
\overline{\mathbf{a}}_{12} &= \mathbf{u}_{12} \\
P_{22} \overline{a}_{11}^{-1} \overline{\mathbf{a}}_{21} &=  \mathbf{\mathit{l}}_{21} \\
\end{aligned}

8. Из полученных элементов восстанавливаем матрицы $\mathbf{L}$, $\mathbf{U}$ и $\mathbf{P}$.

Рассмотрим одно из назначений LU-разложения: решение СЛАУ.
Алгоритм прямой подстановки для решения задачи $\mathbf{L} \mathbf{y} = \mathbf{b}$:

\begin{equation}
\begin{bmatrix}
l_{11} & 0 & ... & 0 \\
l_{21} & l_{22} & ... & 0 \\
\vdots & \vdots & \ddots & \vdots \\
l_{n1} & l_{n2} & ... & l_{nn} \\
\end{bmatrix}
\begin{bmatrix}
y_{1} \\
y_{2} \\
\vdots \\
y_{n} \\
\end{bmatrix} =
\begin{bmatrix}
b_{1} \\
b_{2} \\
\vdots \\
b_{n} \\
\end{bmatrix}
\end{equation}

Решение - путём последовательной подстановки:

\begin{aligned}
y_{1} &= \frac{b_1}{l_{11}} \\
y_{2} &= \frac{b_2 - l_{21}y_{1}}{l_{22}} \\
\vdots& \\
y_{n} &= \frac{b_n - \sum_{j=1}^{n-1}l_{nj}y_{j}}{l_{nn}} \\
\end{aligned}

Ограничения: если в матрице $\mathbf{L}$ хотя бы один диагональный элемент - нуль, то система сингулярна и решений не имеет, в противном случае у неё единственное решение.

In [ ]:
def forward_sub(L, b):
    """x = forward_sub(L, b) - решение уравнения Lx = b
       L - нижняя треугольная матрица (не обязательно с единицами на Г.Д.)
       b - вектор правой части, len(b) == L.shape[0] == L.shape[1]
    """
    n = L.shape[0]
    x = np.zeros(n)
    for i in range(n):
        tmp = b[i]
        for j in range(i):
            tmp -= L[i,j] * x[j]
        x[i] = tmp / L[i,i]
    return x

Вводим алгоритм обратной подстановки для решения задачи $\mathbf{U} \mathbf{x} = \mathbf{y}$:

\begin{equation}
\begin{bmatrix}
u_{11} & u_{12} & ... & u_{1n} \\
0 & u_{22} & ... & u_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & ... & u_{nn} \\
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
\vdots \\
x_{n} \\
\end{bmatrix} =
\begin{bmatrix}
y_{1} \\
y_{2} \\
\vdots \\
y_{n} \\
\end{bmatrix}
\end{equation}

Решение - путём последовательной подстановки:

\begin{aligned}
x_{n} &= \frac{y_n1}{u_{nn}} \\
x_{n-1} &= \frac{y_{n-1} - u_{n-1n}x_{n}}{u_{n-1n-1}} \\
\vdots& \\
x_{1} &= \frac{y_1 - \sum_{j=2}^{n}u_{nj}x_{j}}{u_{nn}} \\
\end{aligned}

Ограничения: как и в случае прямой подстановки: если в матрице $\mathbf{U}$ хотя бы один диагональный элемент - нуль, то система сингулярна и решений не имеет, в противном случае у неё единственное решение.

In [ ]:
def back_sub(U, b):
    """x = back_sub(U, b) - решение уравнения Ux = b
       U - верхняя треугольная матрица
       b - вектор правой части, len(b) == U.shape[0] == U.shape[1]
    """
    n = U.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        tmp = b[i]
        for j in range(i+1, n):
            tmp -= U[i,j] * x[j]
        x[i] = tmp / U[i,i]
    return x

In [ ]:
def lu_decomp(A):
    """
    (L, U) = lu_decomp(A) - разложение A = L U
    """
    n = A.shape[0]
    if n == 1:
        L = np.array([[1]])
        U = A.copy()
        return (L, U)

    A11 = A[0,0]
    A12 = A[0,1:]
    A21 = A[1:,0]
    A22 = A[1:,1:]

    L11 = 1
    U11 = A11

    L12 = np.zeros(n-1)
    U12 = A12.copy()

    L21 = A21.copy() / U11
    U21 = np.zeros(n-1)

    S22 = A22 - np.outer(L21, U12)
    (L22, U22) = lu_decomp(S22)

    if L21.ndim == 1 or L21.shape[1] == 1:
        L21 = L21.reshape((L21.size, 1))
    if L12.ndim == 1:
        L12 = L12.reshape((1, L12.size))

    print(L11, L12.shape, L21.shape, L22.shape)
    L = np.block([[L11, L12], [L21, L22]])

    if U21.ndim == 1 or U21.shape[1] == 1:
        U21 = U21.reshape((U21.size, 1))
    if L12.ndim == 1:
        U12 = U12.reshape((1, U12.size))

    print(U11, U12.shape, U21.shape, U22.shape)
    U = np.block([[U11, U12], [U21, U22]])
    return (L, U)

In [ ]:
def lu_solve(L, U, b):
    """x = lu_solve(L, U, b) решение A x = b: A = LU
       L must be a lower-triangular matrix
       U must be an upper-triangular matrix of the same size as L
       b must be a vector of the same leading dimension as L
    """
#    L, U = lu_decomp(A)
    y = forward_sub(L, b)
    x = back_sub(U, y)
    return x

In [ ]:
def lup_decomp(A):
    """(L, U, P) = lup_decomp(A) is the LUP decomposition P A = L U
       A is any matrix
       L will be a lower-triangular matrix with 1 on the diagonal, the same shape as A
       U will be an upper-triangular matrix, the same shape as A
       P will be a permutation matrix, the same shape as A
    """
    n = A.shape[0]
    if n == 1:
        L = np.array([[1]])
        U = A.copy()
        P = np.array([[1]])
        return (L, U, P)

    i = np.argmax(A[:,0])
    A_bar = np.vstack([A[i,:], A[:i,:], A[(i+1):,:]])

    A_bar11 = A_bar[0,0]
    A_bar12 = A_bar[0,1:]
    A_bar21 = A_bar[1:,0]
    A_bar22 = A_bar[1:,1:]

    S22 = A_bar22 - np.dot(A_bar21, A_bar12) / A_bar11

    (L22, U22, P22) = lup_decomp(S22)

    L11 = 1
    U11 = A_bar11

    L12 = np.zeros(n-1)
    U12 = A_bar12.copy()

    L21 = np.dot(P22, A_bar21) / A_bar11
    U21 = np.zeros(n-1)

    if L21.ndim == 1 or L21.shape[1] == 1:
        L21 = L21.reshape((L21.size, 1))
    if L12.ndim == 1:
        L12 = L12.reshape((1, L12.size))

    if U21.ndim == 1 or U21.shape[1] == 1:
        U21 = U21.reshape((U21.size, 1))
    if L12.ndim == 1:
        U12 = U12.reshape((1, U12.size))

    L = np.block([[L11, L12], [L21, L22]])
    U = np.block([[U11, U12], [U21, U22]])



    P = np.block([
        [np.zeros((1, i-1)), 1,                  np.zeros((1, n-i))],
        [P22[:,:i],      np.zeros((n-1, 1)), P22[:,i:]]
    ])
    return (L, U, P)


In [ ]:
A, b

(array([[ 1,  0,  7],
        [ 0, -1,  0],
        [ 7,  0,  2]]),
 array([4.1, 9.7]))

In [ ]:
def lup_solve(L, U, P, b):
    """x = lup_solve(L, U, P, b) is the solution to L U x = P b
       L must be a lower-triangular matrix
       U must be an upper-triangular matrix of the same shape as L
       P must be a permutation matrix of the same shape as L
       b must be a vector of the same leading dimension as L
    """
    z = np.dot(P, b)
    x = lu_solve(L, U, z)
    return x

In [ ]:
def linear_solve(A, b):
    """x = linear_solve(A, b) is the solution to A x = b (computed with partial pivoting)
       A is any matrix
       b is a vector of the same leading dimension as A
       x will be a vector of the same leading dimension as A
    """
    (L, U, P) = lup_decomp(A)
    x = lup_solve(L, U, P, b)
    return x

In [ ]:
A = np.array([[6, 1], [2, 4]])
b = np.array([4.1, 9.7])

In [ ]:
A = np.array([[6, 1, 4], [2, 4, 1], [10, 11, 5]])
b = np.array([4.1, 9.7, 2.1])

In [ ]:
np.block([[1, np.array([2, 2]).reshape((1, 2))], [np.array([3, 3]).reshape((2, 1)), np.array([4, 4, 4, 4]).reshape((2, 2))]])

array([[1, 2, 2],
       [3, 4, 4],
       [3, 4, 4]])

In [ ]:
L, U = lu_decomp(A)

1 (1, 1) (1, 1) (1, 1)
3.6666666666666665 (1,) (1, 1) (1, 1)
1 (1, 2) (2, 1) (2, 2)
6 (2,) (2, 1) (2, 2)


In [ ]:
print(L)
print(U)
print(np.dot(L, U))

[[1.         0.         0.        ]
 [0.33333333 1.         0.        ]
 [1.66666667 2.54545455 1.        ]]
[[ 6.          1.          4.        ]
 [ 0.          3.66666667 -0.33333333]
 [ 0.          0.         -0.81818182]]
[[ 6.  1.  4.]
 [ 2.  4.  1.]
 [10. 11.  5.]]


Для решения СЛАУ вида $Ax = b$ мы раскладываем матрицу $A$:

\begin{aligned}
Ax &= b \\
LUx &= b \\
Ux &= L^{-1} b \\
x &= U^{-1}(L^{-1} b)
\end{aligned}

Для вычисления $L^{-1} b$ мы используем прямую подстановку, для вычисления $x = U^{-1}(L^{-1} b)$ - используется обратная подстановка.

In [ ]:
linear_solve(A, b)

array([0.70019608, 0.16862745, 0.16862745])